In [12]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [13]:
df = pd.read_csv("data.csv")
df.head()

,review,sentiment
0,Every great gangster movie has under-currents ...,positive
1,"I just saw this film last night, and I have to...",positive
2,This film is mildly entertaining if one neglec...,negative
3,Quentin Tarantino's partner in crime Roger Ava...,negative
4,I sat through this on TV hoping because of the...,negative


In [14]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [15]:
import nltk

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sandeep\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sandeep\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sandeep\AppData\Roaming\nltk_data...


True

In [16]:
df = normalize_text(df)
df.head()

,review,sentiment
0,every great gangster movie under current human...,positive
1,saw film last night say loved every minute tak...,positive
2,film mildly entertaining one neglect acknowled...,negative
3,quentin tarantino s partner crime roger avary ...,negative
4,sat tv hoping name would worth time but dear g...,negative


In [17]:
df['sentiment'].value_counts()

sentiment
negative    269
positive    231
Name: count, dtype: int64

In [18]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [19]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
0,every great gangster movie under current human...,1
1,saw film last night say loved every minute tak...,1
2,film mildly entertaining one neglect acknowled...,0
3,quentin tarantino s partner crime roger avary ...,0
4,sat tv hoping name would worth time but dear g...,0


In [20]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [21]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [23]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/kvsandeep860/Mlops_end_to_end.mlflow')
dagshub.init(repo_owner='kvsandeep860', repo_name='Mlops_end_to_end', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

c:\Users\sandeep\miniconda4\envs\atlas\lib\site-packages\rich\live.py:260: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=cce82bd6-828a-44ec-ac7c-d7fb692a70fb&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=14ea5c2115dced2d2707fd9824154255ea7d88fa2cdd050dbb9d6196070b4d9c




Accessing as kvsandeep860

Initialized MLflow to track repo "kvsandeep860/Mlops_end_to_end"

Repository kvsandeep860/Mlops_end_to_end initialized!

2026/02/08 22:34:52 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/9b6d53a08cb648c197f7b2281ee35fc8', creation_time=1770570296985, experiment_id='0', last_update_time=1770570296985, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [24]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2026-02-08 22:35:25,178 - INFO - Starting MLflow run...
2026-02-08 22:35:26,210 - INFO - Logging preprocessing parameters...
2026-02-08 22:35:27,670 - INFO - Initializing Logistic Regression model...
2026-02-08 22:35:27,671 - INFO - Fitting the model...
2026-02-08 22:35:27,713 - INFO - Model training complete.
2026-02-08 22:35:27,713 - INFO - Logging model parameters...
2026-02-08 22:35:28,107 - INFO - Making predictions...
2026-02-08 22:35:28,108 - INFO - Calculating evaluation metrics...
2026-02-08 22:35:28,129 - INFO - Logging evaluation metrics...
2026-02-08 22:35:30,099 - INFO - Saving and logging the model...
2026/02/08 22:35:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\sandeep\miniconda4\envs\atlas\lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which c

🏃 View run traveling-goose-275 at: https://dagshub.com/kvsandeep860/Mlops_end_to_end.mlflow/#/experiments/0/runs/7860163b032e41e696dec80400d2b1bc
🧪 View experiment at: https://dagshub.com/kvsandeep860/Mlops_end_to_end.mlflow/#/experiments/0
